In [54]:
import gtfs_kit as gk
import os.path
import pandas as pd
import geopandas as gpd
import geoplot as gplt
import datetime as dt
import gtfs_functions as gtfs

In [2]:
gtfs_dir = "/mnt/c/Users/maita.schade/Nextcloud/Documents/Work/Gap_Map/raw/gtfs"
years = [2020, 2021]
paths = {y: os.path.join(gtfs_dir,str(y),str(y)+".zip") for y in years}
#feeds = {y: gk.read_feed(paths[y], dist_units='km') for y in years}

1) Cut by time

need dates to cut by...

In [34]:
window = ('20200301','20200331')

In [4]:
gtf = gk.read_feed(paths[2020], dist_units='km')

/home/mschade/anaconda3/envs/gapmapbox/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3437: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
gtf.calendar_dates.date[1] < gtf.calendar.start_date[1]

False

In [10]:
gtf.calendar.start_date[1]

'20191215'

In [11]:
gtf.calendar_dates.date[1]

'20191218'

In [78]:
type(gtf.calendar_dates.merge(gtf.calendar, on="service_id", how='outer').loc[272249].exception_type)

numpy.float64

Calculate the service windows

In [69]:
"20200101" > "nan"

False

In [99]:
~pd.isna(s)

0    False
1    False
dtype: bool

In [100]:
def first(s):
    s[0]

def safeMax(s):
    if all(pd.isna(s)):
        return pd.NA
    else:
        return s[~pd.isna(s)].max()
    
def safeMin(s):
    if all(pd.isna(s)):
        return pd.NA
    else:
        return s[~pd.isna(s)].min()
    

In [101]:
service_windows = gtf.calendar_dates.merge(gtf.calendar, on="service_id", how='outer').groupby("service_id").agg(
            min_exc=('date', safeMin),
            max_exc=('date', safeMax),
            start_date = ("start_date", 'first'),
            end_date = ("end_date", 'first')
    ).reset_index()

In [102]:
service_windows

,service_id,min_exc,max_exc,start_date,end_date
0,1,20191216,20201118,20200106,20201209
1,10,20191216,20201118,20200203,20201211
2,100,20200225,20201030,20200908,20200911
3,1000,20191220,20200611,20191216,20201211
4,10000,20200224,20200611,20200102,20200612
...,...,...,...,...,...
24767,9995,<NA>,<NA>,20200829,20201212
24768,9996,20191216,20200214,20200106,20200228
24769,9997,20191225,20200101,20191225,20200328
24770,9998,20200619,20201004,20200615,20201212


In [104]:
service_windows['first'] = service_windows.loc[:,'min_exc':'end_date'].apply(safeMin, axis=1) 

service_windows['last'] = service_windows.loc[:,'min_exc':'end_date'].apply(safeMax, axis=1)

service_windows = service_windows[['service_id','first','last']]

I don't think it actually makes sense to treat calendar and service together... what happens to an exception on an off day?
Unfortunately, in the constellation we have here, every exception is supposed to belong to a specific service referenced in calendar

In [107]:
gtf.calendar.service_id.isin(gtf.calendar_dates.service_id)

0         True
1        False
2         True
3         True
4         True
         ...  
23732    False
23733    False
23734    False
23735    False
23736    False
Name: service_id, Length: 23737, dtype: bool

In [110]:
(~gtf.calendar_dates.service_id.isin(gtf.calendar_dates.service_id)).sum()

0

Pick applicable services

In [37]:
services = service_windows[
    (service_windows['first'] < window[1]) & (service_windows['last'] > window[0])
    ].service_id

Filter calendar, calendar_dates by service

In [38]:
calendar = gtf.calendar[gtf.calendar.service_id.isin(services)]

In [40]:
calendar_dates = gtf.calendar_dates[gtf.calendar_dates.service_id.isin(services)]

Adjust dates so max, min are ok

First, the calendar itself

In [48]:
calendar.loc[23716,:]

monday               1
tuesday              1
wednesday            1
thursday             1
friday               1
saturday             1
sunday               1
start_date    20201109
end_date      20201212
service_id        4142
Name: 23716, dtype: object

In [ ]:
calendar_dates

In [46]:
def elementMax(s, e):
    return s.apply(lambda e1: max(e1, e))

In [47]:
elementMax(calendar.start_date, window[0])

0        20200301
2        20200301
7        20200301
9        20200301
10       20200301
           ...   
23633    20200925
23676    20201101
23678    20201101
23705    20201107
23716    20201109
Name: start_date, Length: 13456, dtype: object